# OpenAI 快速入門

# 概述  
大型語言模型 (Large language models,LLM) 會嘗試依據用戶輸入的一串文字預測接下來接續出現的內容，本範例將展示 LLM 概念，並以 Python 3.11 與 OpenAI 函式庫，展示提示設計 (prompt design) 與 Azure OpenAI Service 主要功能。

有關更多的資訊，請參閱微軟官方 Azure Open AI 快速入門線上文件 https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio

### 開始使用 Azure OpenAI Service

新客戶需要 [申請核准](https://aka.ms/oai/access) 後才能夠使用 Azure OpenAI Service
當申請核准後，用戶可以登入 Azure portal 建立 Azure OpenAI Service 資源，並開始探索 Azure AI Studio 各項功能。

[2023/1/19 Azure OpenAI Service 正式發表時所整理之服務介紹](https://techcommunity.microsoft.com/t5/educator-developer-blog/azure-openai-is-now-generally-available/ba-p/3719177 )


### 建立第一個提示 API 呼叫
以下將做一個簡單的練習，示範基本的提示設計 (prompts design) 與自動完成 (completion) API 呼叫。  

**步驟**:  
1. 安裝 OpenAI 所需套件
2. 運用輔助函式庫，從環境變數取得 Azure OpenAI API 鍵值
3. 選擇適當之模型
4. 提示設計 (Prompt Design)
5. 呼叫 API

### 1. 安裝 OpenAI 所需套件

In [17]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


### 2. 運用輔助函式庫，從環境變數取得 Azure OpenAI API 鍵值

In [1]:
import os
import openai
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()

# 設定呼叫 OpenAI API 所需連線資訊
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "發生錯誤: 缺少 Azure OpenAI Service 鍵值"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")
assert RESOURCE_ENDPOINT, "發生錯誤: 缺少 Azure OpenAI Service 連線端點"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "發生錯誤: Azure OpenAI Service 連線端點格式應該如: \n\n\t<您所建立之資源名稱>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

### 3. 選擇適當之模型  
Azure OpenAI Service 提供多種不同功能與價位的模型提供用戶選用。 可選用的模型可能會依區域資料中心而有所不同。 GPT-3 等較舊的模型將於 2024 年 7 月淘汰，搭配 GPT-3 的 Completion API 也將逐漸被 ChatCompletion API 所取代。目前建議以 ChatCompletion API 搭配使用以下的模型 :

* gpt-4
* gpt-4-32k
* gpt-35-turbo
* gpt-35-turbo-16k
* dalle2 (技術預覽) 

由於在 2023 年的過渡階段，本 Quick Start 仍包含了 Completion API 內容與少數 GPT-3 模型的範例。模型支援最新資訊可參閱微軟官網 [Azure OpenAI Service 模型資訊](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)  

此外過去 GPT-3 提供三種模型 Embeddings 模型可用於

* 文字近似比對 (Similarity)
* 文字搜尋 (Text Search)
* 程式碼搜尋 (Code Search)

多種較舊的 Embeddings 模型同樣將於 2024 年 7 月淘汰。目前建議使用的 Embedding 模型為

* text-embedding-ada-002

此模型能夠以單一模型涵蓋支援三種情境。

In [2]:
# 選取自動完成 (Completion) API 所使用的模型

model = os.getenv('DEPLOYMENT_NAME')


## 4. 提示設計 (Prompt Design)

大型語言模型的神奇之處；在於能夠給予少量的範例量即可理解人類的自然語言意義，並利用提示 (Prompt) 操控模型產出有用的文字內容，例如

* 如何拼字
* 文法調整
* 以不同的方式改寫文案
* 回答問題
* 進行對話
* 多種人類語言之間的翻譯
* 程式碼撰寫
* etc.

#### 如何操控大型語言模型

提示 (Prompts) 是一段文字用以操控大型語言模型，可以透過以下幾種方式提示大型語言模型產生我們期望之輸出:
+ 說明 (Instruction)：告訴模型你想要什麼
+ 完成 (Completion)：誘導模型完成與填補您想要的文字
+ 演示 (Demonstration)：以範例的方式向模型展示您想要產出的文字內容，可分為三類 :
    - 無樣本學習 (Zero-shot) – 完全不用給範例直接讓模型預測應該生成的內容
    - 單一樣本學習 (One-shot) – 僅給一個範例示範給模型，接下來讓模型去預測應該生成的內容
    - 少量樣本學習 (Few-shot) – 給模型多個範例後，接下來讓模型去預測應該生成的內容


#### 三個建立提示 (prompts) 的準則:

**展示與講述 (Show and tell)**. 如前所述透過說明 (Instruction) 與展示 (Demonstration) 的結合；明確的告訴模型您想要產出什麼內容。例如您希望模型按字母順序列表排序，或是依照文字內含的情感來進行段落分類，請在提示中明確經精準的表達出來。

**提供高品質的提示資料 (Provide quality data)**. 如果您嘗試利用 Azure OpenAI Service 遵循某種模式自動替文字內容分類 (Classify) 與貼標籤，請確保提示中提供了足夠的範例。並且校對您給的範例是否有錯別字，或是不正確的分類。大型語言模型通常夠聰明能夠理解拼字與錯別字背後的意義，但也可能誤以為這是故意的，影響了之後文字分類的正確性。

**檢查參數設定 (Check your settings)**  temperature 和 top_p 兩個參數可以控制模型產出的文字內容的多樣性。 如果您要求模型產出一個答案，那麼您需要將這些參數值設置的較低。 如果您希望產出更多樣化的文字內容，則需要將這兩個參數設定的更高。這兩個參數與控制模型的 “聰明” 或 “創造力” 無關，僅是控制產出內容更是更單一或是有著更豐富的變化。

使用 Azure AI Studio 的 Playground 可以驗證提示是否產出您期望的文字內容以及確認參數設定的效果。

資料來源: https://platform.openai.com/docs/quickstart

### 5. 呼叫 API

In [4]:
# 建立第一個提示 (prompt)，英文書寫中是必須該使用 Oxford comma (https://zh.wikipedia.org/wiki/%E7%89%9B%E6%B4%A5%E9%80%97%E8%99%9F)?
text_prompt = "Should oxford commas always be used?"

In [5]:
# 最簡之 API 呼叫
openai.Completion.create(
    engine=model,
    prompt=text_prompt,
    max_tokens=60
)

<OpenAIObject text_completion id=cmpl-7jGybIvC5mAvxFCeMG1GLZ9jnYXUI at 0x2bef46adaf0> JSON: {
  "id": "cmpl-7jGybIvC5mAvxFCeMG1GLZ9jnYXUI",
  "object": "text_completion",
  "created": 1691024373,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": "\n\nThe Oxford comma is the comma used before the conjunction in a list.\n\nIt can a",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 8,
    "total_tokens": 26
  }
}

### 重複呼叫，產生出來的結果會相同嗎?

In [6]:
openai.Completion.create(
    engine=model,
    prompt=text_prompt,
    max_tokens=60
)

<OpenAIObject text_completion id=cmpl-7jGyiNwpFR5XAR55Z0lZX3yAz7vfl at 0x2bef46adf70> JSON: {
  "id": "cmpl-7jGyiNwpFR5XAR55Z0lZX3yAz7vfl",
  "object": "text_completion",
  "created": 1691024380,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": " For instance,\n\nAlpha, Beta, and Gamma\nAlpha, Beta and Gamma\n\nThe second example is incorrect, but are there instances where it is required? Or is it purely a style issue?\nGesteenjnvso 2018-10-19: The use of a serial comma",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 60,
    "prompt_tokens": 8,
    "total_tokens": 68
  }
}

# 使用案例
1. 內容摘要 (Summarize Text)
2. 文字內容分類 (Classify)，貼標籤  
3. 產生新產品英文名稱
4. 內嵌 Embeddings


## 內容摘要 (Summarize Text)

在一段文章之後加入提示 TL;DR 即可引導模型來進行內容摘要。TL;DR 是英文 too long; didn't read，我們也可以在 TL;DR 後面增添額外提示，例如 "TL;DR 2 sentence" 將所有內容以兩句話總結出來。本範例是採用無樣本學習 (Zero-shot) 意即完全不用給範例直接讓模型產出摘要的內容，為了避免 TL;DR 讓模型以英文回覆摘要內容，我們給予以下提示 :

本文:西班牙選舉結束，變數卻比選前更多。西班牙國會大選已在7月24日開票確定結果，右翼在野黨人民黨贏得136個席次，勝過首相桑切斯（Pedro Sánchez）所屬左派社會黨贏得122席，但雙方與其盟友加總，皆無法獲得國會過半所需的176席——這意味著，西班牙新任政府的組成依然懸而未決。原本選前人民黨來勢洶洶，志在將桑切斯趕下台，儘管人民黨黨魁費侯（Alberto Núñez Feijóo）有機會上位，桑切斯仍有不小的機會繼續執政。然而對同樣陷入未過半僵局的社會黨來說，保住政權勢必付出代價——為了獲得更多議員的支持，社會黨勢必要尋求與加泰隆尼亞或是巴斯克獨派政黨合作，而條件可能是必須允許獨立公投。

內容摘要:


In [7]:
prompt = "本文:西班牙選舉結束，變數卻比選前更多。西班牙國會大選已在7月24日開票確定結果，右翼在野黨人民黨贏得136個席次，" \
         "勝過首相桑切斯（Pedro Sánchez）所屬左派社會黨贏得122席，但雙方與其盟友加總，皆無法獲得國會過半所需的176席——這意味著，" \
         "西班牙新任政府的組成依然懸而未決。原本選前人民黨來勢洶洶，志在將桑切斯趕下台，儘管人民黨黨魁費侯（Alberto Núñez Feijóo）有機會上位，" \
         "桑切斯仍有不小的機會繼續執政。然而對同樣陷入未過半僵局的社會黨來說，保住政權勢必付出代價——為了獲得更多議員的支持，" \
         "社會黨勢必要尋求與加泰隆尼亞或是巴斯克獨派政黨合作，而條件可能是必須允許獨立公投。\n" \
         "內容摘要:"

model = os.getenv('DEPLOYMENT_NAME')

In [8]:
# 可透過參數值控制模型的行為，例如希望摘要內容不要過長，可以透過 max_tokens 參數來控制產出的 Token 數量
openai.Completion.create(
  engine=model,
  prompt=prompt,
  temperature=0.4,
  max_tokens= 100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)["choices"][0]["text"].strip(" \n")


'西班牙選舉結束，變數卻比選前更多。西班牙國會大選已在7月24日開票確定結果，右翼在野黨人民黨贏得136個席次，勝過首相桑切斯（Pedro Sánchez）所屬左派'

## 文字內容分類 (Classify)，貼標籤

由大型語言模型來做文字分類與貼標籤是常見的應用情境，並用單一樣本學習 (One-shot) 的方式提供一個範例讓模型了解如何貼標籤，並且利用範例讓模型產生出 \#\#\# ，搭配設定 stop sequence，當產出 \#\#\# 字串後就讓模型停止運作 ，提示如下:

標籤分類為以下其中一種 :  Pricing,Support Center Location,Hardware Support,Software Support

客戶詢問: 您好，最近我的筆記型電腦鍵盤上一個按鍵脫落損壞了，我需要維修替換，請問你們有哪些維修據點?

標籤分類: Hardware Support,Support Center Location

\#\#\#

客戶詢問: 我安裝了新的 Linux 顯示卡驅動程式後畫面看不見了 ? 買新的電腦要多少錢?

標籤分類: 


In [9]:
prompt = "標籤分類為以下其中一種 :  Pricing,Support Center Location,Hardware Support,Software Support\n" \
         "客戶詢問: 您好，最近我的筆記型電腦鍵盤上一個按鍵脫落損壞了，我需要維修替換，請問你們有哪些維修據點?\n" \
         "標籤分類: Hardware Support,Support Center Location\n" \
         "###\n" \
         "客戶詢問: 我不知道該如何安裝 Linux 顯示卡驅動程式? 買新的電腦要多少錢?\n" \
         "標籤分類:"

model = os.getenv('DEPLOYMENT_NAME')

In [11]:
response = openai.Completion.create(
  engine=model,
  prompt=prompt,
  temperature=0,
  max_tokens=200,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop="###")

print(response)

{
  "id": "cmpl-7iYSAg6UNtWxkXTTIIFGIx81bDCHz",
  "object": "text_completion",
  "created": 1690853226,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": " Pricing,Software Support\n",
      "index": 0,
      "finish_reason": "stop",
      "logprobs": null
    }
  ],
  "usage": {
    "completion_tokens": 5,
    "prompt_tokens": 183,
    "total_tokens": 188
  }
}


## 產生新產品英文名稱

根據提示的英文單詞來發想出產品名稱。此範例中的提示 (Prompt) 包含了產品基本功能描述的資訊以及多個展現產品特質的英文單字，並用單一樣本學習 (One-shot) 的方式提供一個家用奶昔機的產品命名範例示範給模型看，接下來讓模型去預測應該生成的內容，此外我們還將參數值 Temperature 設的較高 (0.8)，以增加隨機性與更具創新性的回應。提示內容如下:

產品描述: A home milkshake maker

Seed words: fast, healthy, compact.

建議產品名稱: HomeShaker, Fit Shaker, QuickShake, Shake Maker

產品描述: A pair of shoes that can fit any foot size.

Seed words: adaptable, fit, omni-fit.

In [12]:
prompt = "產品描述: A home milkshake maker\n" \
         "Seed words: fast, healthy, compact.\n" \
         "建議產品名稱: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\n" \
         "產品描述: A pair of shoes that can fit any foot size.\n" \
         "Seed words: adaptable, fit, omni-fit.\n"

model = os.getenv('DEPLOYMENT_NAME')

In [13]:
openai.Completion.create(
  engine= model,
  prompt=prompt,
  temperature=0.8,
  max_tokens=25,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)["choices"][0]["text"].strip(" \n")

'建議產品名稱: FitFeet, OmniFit, AnyFit, AdaptFit'

## 內嵌 Embeddings
內嵌 (Embeddings) 是被用於自然語言處理中語言模型的一種技術，可將每個單詞或詞組映射為一組浮點數所代表的向量 (vectors)，用以衡量單詞或字串間的相關性
 * 兩組浮點數之間數字越大代表距離越遠，單詞或字串彼此間的相關性越低
 * 兩組浮點數之間數字越小代表距離越近，單詞或字串彼此間的相關性越高

內嵌 (Embeddings) 可被用於文字之
 * 搜尋 : 搜尋結果依據查詢後字串的相關性排名
 * 聚類 (Clustering) 分析 : 依據字串彼此間相似性分組
 * 建議 :  依據字串間的相關性提出建議
 * 異常檢測 : 辨識出相關性不大的異常值
 * 多樣性測量 (Diversity measurement) : 依據向量分布來分析相似性
 * 分類 (Classification)  :  以最相似的標籤將字串加以分類


In [16]:
# 安裝所需之核心函式庫
%pip install pandas plotly scikit-learn matplotlib plotly_express 


Note: you may need to restart the kernel to use updated packages.


In [10]:
from openai.embeddings_utils import get_embedding, cosine_similarity

In [12]:
text = 'the quick brown fox jumped over the lazy dog'
model = 'text-embedding-ada-002'

In [13]:
openai.Embedding.create(
    input=text, engine=model
)["data"][0]["embedding"]

[-0.004474656656384468,
 0.00978652760386467,
 -0.014904950745403767,
 -0.006424985360354185,
 -0.01135313231498003,
 0.015513833612203598,
 -0.02372107096016407,
 -0.016414472833275795,
 -0.0158182755112648,
 -0.029632311314344406,
 0.021298224106431007,
 0.021095262840390205,
 0.018570933490991592,
 0.004170214757323265,
 -0.0007155169150792062,
 -0.007579326163977385,
 0.02521790750324726,
 -0.004214612767100334,
 0.011175542138516903,
 -0.008587788790464401,
 -0.009513798169791698,
 0.021577294915914536,
 -0.005993693135678768,
 -0.008257976733148098,
 0.006041261833161116,
 0.013040246441960335,
 0.007439790293574333,
 -0.0035169341135770082,
 -0.008955655619502068,
 0.0011939817341044545,
 0.00666600139811635,
 0.0038657733239233494,
 -0.039272960275411606,
 -0.002559211803600192,
 -0.012761174701154232,
 -0.0217422004789114,
 -0.0037072100676596165,
 -0.010458835400640965,
 0.02597901225090027,
 -0.0456916019320488,
 0.009399632923305035,
 0.015653369948267937,
 -0.0226174704730

In [20]:

# 比較多種字句之間的相似程度
automobile_embedding            = openai.Embedding.create(input='汽車', engine=model)["data"][0]["embedding"]
transportation_embedding        = openai.Embedding.create(input='交通工具', engine=model)["data"][0]["embedding"]
description_embedding           = openai.Embedding.create(input='汽車通常有四個輪子，本身具有動力得以驅動前進，不須依軌道或電纜即可行駛，可以讓人們在不同地點之間移動，並且可以運輸物品', engine=model)["data"][0]["embedding"]
dinosaur_embedding              = openai.Embedding.create(input='暴龍', engine=model)["data"][0]["embedding"]
stick_embedding                 = openai.Embedding.create(input='棍子', engine=model)["data"][0]["embedding"]
house_embedding                 = openai.Embedding.create(input='房屋', engine=model)["data"][0]["embedding"]
airplane_embedding              = openai.Embedding.create(input='飛機', engine=model)["data"][0]["embedding"]
automobile_english_embedding    = openai.Embedding.create(input='automobile', engine=model)["data"][0]["embedding"]
automobile_korean_embedding     = openai.Embedding.create(input='자동차', engine=model)["data"][0]["embedding"]
chechnya_embedding              = openai.Embedding.create(input='車臣', engine=model)["data"][0]["embedding"]
chechnya_english_embedding      = openai.Embedding.create(input='chechnya', engine=model)["data"][0]["embedding"]
chechnya_description_embedding  = openai.Embedding.create(input='車臣共和國，是俄羅斯的一個共和國。它位於東歐北高加索地區，靠近里海。該共和國是北高加索聯邦區的一部分，南部與格魯吉亞州接壤。東、北、西分別是達吉斯坦、印古什和俄羅斯北奧塞梯-阿拉尼亞共和國。西北部是斯塔夫羅波爾邊疆區', engine=model)["data"][0]["embedding"]
maserati_description_embedding  = openai.Embedding.create(input='瑪莎拉蒂送往生者最後一程確實很尊榮，禮儀業者表示目前這款很少改裝使用，有家屬為了讓往生者體面的離開，會選擇好一點的款式', engine=model)["data"][0]["embedding"]

print ("automobile_embedding vs automobile_embedding")
print(cosine_similarity(automobile_embedding,automobile_embedding))
print ("automobile_embedding vs transportation_embedding")
print(cosine_similarity(automobile_embedding, transportation_embedding))
print ("automobile_embedding vs description_embedding")
print(cosine_similarity(automobile_embedding, description_embedding))
print ("automobile_embedding vs dinosaur_embedding")
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print ("automobile_embedding vs stick_embedding")
print(cosine_similarity(automobile_embedding, stick_embedding))
print ("automobile_embedding vs house_embedding")
print(cosine_similarity(automobile_embedding, house_embedding))
print ("automobile_embedding vs airplane_embedding")
print(cosine_similarity(automobile_embedding, airplane_embedding))
print ("automobile_embedding vs automobile_english_embedding")
print(cosine_similarity(automobile_embedding, automobile_english_embedding))
print ("automobile_embedding vs automobile_korean_embedding")
print(cosine_similarity(automobile_embedding, automobile_korean_embedding))
print ("automobile_embedding vs chechnya_embedding")
print(cosine_similarity(automobile_embedding,chechnya_embedding ))
print ("automobile_embedding vs chechnya_english_embedding")
print(cosine_similarity(automobile_embedding,chechnya_english_embedding ))
print ("automobile_embedding vs chechnya_description_embedding")
print(cosine_similarity(automobile_embedding,chechnya_description_embedding ))
print ("automobile_embedding vs maserati_description_embedding")
print(cosine_similarity(automobile_embedding,maserati_description_embedding  ))


automobile_embedding vs automobile_embedding
1.0000000000000002
automobile_embedding vs transportation_embedding
0.8590042675444729
automobile_embedding vs description_embedding
0.8474689758064199
automobile_embedding vs dinosaur_embedding
0.7916123902327199
automobile_embedding vs stick_embedding
0.7932191675248061
automobile_embedding vs house_embedding
0.8263383247226136
automobile_embedding vs airplane_embedding
0.8689659513585208
automobile_embedding vs automobile_english_embedding
0.8848820675646012
automobile_embedding vs automobile_korean_embedding
0.8867113614343675
automobile_embedding vs chechnya_embedding
0.852131766183046
automobile_embedding vs chechnya_english_embedding
0.7303717474355156
automobile_embedding vs chechnya_description_embedding
0.7651123686975985
automobile_embedding vs maserati_description_embedding
0.7621811716929773


## 比較 CNN 每日新聞資料集之文章相似程度
資料來源: https://huggingface.co/datasets/cnn_dailymail


In [9]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"' 
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()                      

,articles,highligths
0,"BREMEN, Germany -- Carlos Alberto, who scored ...",Werder Bremen pay a club record $10.7 million ...
1,"(CNN) -- Football superstar, celebrity, fashio...",Beckham has agreed to a five-year contract wit...
2,"LOS ANGELES, California (CNN) -- Youssif, the ...","Boy on meeting Spider-Man: ""It was my favorite..."


In [10]:
article1_embedding    = openai.Embedding.create(input=cnn_df.articles.iloc[0], engine=model)["data"][0]["embedding"]
article2_embedding    = openai.Embedding.create(input=cnn_df.articles.iloc[1], engine=model)["data"][0]["embedding"]
article3_embedding    = openai.Embedding.create(input=cnn_df.articles.iloc[2], engine=model)["data"][0]["embedding"]

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

0.7621254342360414
0.7103234824922888


# 參考資料  

1 - [Openai Cookbook](https://github.com/openai/openai-cookbook)  
2 - [Azure Documentation - Azure Open AI Models](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/models)  
3 - [Azure AI Studio Examples](https://oai.azure.com/portal)  
4 - [Best practices for fine-tuning GPT-3 to classify text](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#)

# 更多來自微軟協助  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)  
AI 專長雲端架構師 [aka.ms/airangers](aka.ms/airangers)

# 原始貢獻者
* Brandon Cowen
* Ashish Chauhun
* Louis Li  
